## Apache Spark MLLib
Let's return to the farmer's market dataset and use Spark to explore the hypothesis:
* The number of farmer's markets in a given zip code can be predicted from the income and taxes paid in a given area.

There are serveral steps to this process:
1. **Part One - Load and prepare the data** 
  * Verify and/or load table data
  * Prepare the data by aggregating, grouping and counting table data values
  * Join data from the prepared tables
  * Convert 'null' values in the joined data to '0'
2. **Part Two - Use the Spark ML Library**
  * Create and display a vector with the the features you'd like to explore in a scatterplot
  * Split the dataset into testing and training sets, cache both and call an action to load the cache
  * Create a linear regression model and fit the model with your training data
  * Use your model by calling predict on it
  * Evaluate and update your model  
  * Train and use the most optimal model

### Part One - Load and Prepare the data 
* Load the table `cleaned_taxes` into a dataframe (created in previous exercise)

In [3]:
cleanedTaxes = sqlContext.sql("SELECT * FROM cleaned_taxes")
cleanedTaxes.show()

+-----+-------+--------------+-------------+------+-------------------+-----------------------+-----------------+--------------+
state|zipcode|single_returns|joint_returns|numdep|total_income_amount|taxable_interest_amount|net_capital_gains|biz_net_income|
+-----+-------+--------------+-------------+------+-------------------+-----------------------+-----------------+--------------+
 AL| 0| 488030| 122290|571240| 1.1444868E7| 77952.0| 23583.0| 824487.0|
 AL| 0| 195840| 155230|383240| 1.7810952E7| 81216.0| 54639.0| 252768.0|
 AL| 0| 72710| 146880|189340| 1.6070153E7| 80627.0| 84137.0| 259836.0|
 AL| 0| 24860| 126480|134370| 1.4288572E7| 71086.0| 105947.0| 214668.0|
 AL| 0| 16930| 168170|177800| 2.605392E7| 149150.0| 404166.0| 567439.0|
 AL| 0| 3530| 42190| 48270| 2.0752068E7| 271416.0| 1569967.0| 822565.0|
 AL| 3500| 950| 260| 710| 19851.0| 183.0| 4.0| 1657.0|
 AL| 3500| 590| 410| 860| 49338.0| 172.0| 54.0| 788.0|
 AL| 3500| 290| 490| 620| 56170.0| 185.0| 139.0| 584.0|
 AL| 3500| 90| 490| 530| 52977.0| 89.0| 173.0| 339.0|
 AL| 3500| 40| 460| 450| 64329.0| 205.0| 709.0| 1720.0|
 AL| 3500| 0| 40| 30| 15359.0| 130.0| 0.0| 0.0|
 AL| 3500| 800| 190| 860| 19011.0| 135.0| -1.0| 840.0|
 AL| 3500| 360| 300| 850| 36996.0| 87.0| 42.0| -193.0|
 AL| 3500| 140| 230| 340| 28664.0| 95.0| 22.0| 233.0|
 AL| 3500| 50| 170| 200| 19583.0| 38.0| 62.0| -48.0|
 AL| 3500| 0| 160| 170| 25136.0| 67.0| 48.0| 84.0|
 AL| 3500| 0| 0| 0| 0.0| 0.0| 0.0| 0.0|
 AL| 3500| 250| 110| 200| 5515.0| 26.0| 0.0| 543.0|
 AL| 3500| 110| 150| 230| 11906.0| 32.0| 0.0| 188.0|
+-----+-------+--------------+-------------+------+-------------------+-----------------------+-----------------+--------------+
only showing top 20 rows

NOTE: If the table did NOT load, then run the next couple of cells to re-load the data.

In [5]:
# taxes2013 = spark.read\
#   .option("header", "true")\
#   .csv("dbfs:/databricks-datasets/data.gov/irs_zip_code_data/data-001/2013_soi_zipcode_agi.csv")
# taxes2013.createOrReplaceTempView("taxes2013")

In [6]:
# %sql
# DROP TABLE IF EXISTS cleaned_taxes;

# CREATE TABLE cleaned_taxes AS
# SELECT 
#   state, 
#   int(zipcode / 10) as zipcode,
#   int(mars1) as single_returns,
#   int(mars2) as joint_returns,
#   int(numdep) as numdep,
#   double(A02650) as total_income_amount,
#   double(A00300) as taxable_interest_amount,
#   double(a01000) as net_capital_gains,
#   double(a00900) as biz_net_income
# FROM taxes2013

* Load the market dataset to a permanent table named `markets`

In [8]:
markets = spark.read\
  .option("header", "true")\
  .csv("dbfs:/databricks-datasets/data.gov/farmers_markets_geographic_data/data-001/market_data.csv")

* Use `sum` to aggreggate all the columns in the `cleanedTaxes` dataset -- NOTE: Some data will be nonsense (i.e.summing zipcode) but other data could become useful features (i.e. summing AGI in the zipcode).
* Group the `cleanedTaxes` dataframe by zipcode, then `sum` to aggregate across all columns. 
* Save the resulting dataframe in `summedTaxes`
* `show` the `summedTaxes` dataframe

In [10]:
summedTaxes = cleanedTaxes\
  .groupBy("zipcode")\
  .sum()
  
summedTaxes.show()

+-------+------------+-------------------+------------------+-----------+------------------------+----------------------------+----------------------+-------------------+
zipcode|sum(zipcode)|sum(single_returns)|sum(joint_returns)|sum(numdep)|sum(total_income_amount)|sum(taxable_interest_amount)|sum(net_capital_gains)|sum(biz_net_income)|
+-------+------------+-------------------+------------------+-----------+------------------------+----------------------------+----------------------+-------------------+
 8592| 257760| 2390| 2950| 4430| 283765.0| 1486.0| 3844.0| 8238.0|
 7240| 86880| 12980| 12170| 22710| 1680668.0| 11702.0| 43462.0| 70650.0|
 7253| 391662| 990| 1570| 1730| 116903.0| 657.0| 1163.0| 3764.0|
 8086| 145548| 2160| 2890| 3150| 362196.0| 2051.0| 7147.0| 14109.0|
 3175| 57150| 4250| 4440| 9490| 495720.0| 4784.0| 7824.0| 14127.0|
 4818| 260172| 80270| 59790| 99040| 8578999.0| 37205.0| 72843.0| 183436.0|
 4900| 205800| 36180| 22850| 37820| 3678795.0| 27176.0| 114883.0| 119738.0|
 3918| 70524| 8440| 6370| 17740| 1000394.0| 4644.0| 19025.0| 15917.0|
 6336| 266112| 25750| 28230| 42750| 4171419.0| 18456.0| 60965.0| 93342.0|
 6466| 193980| 590| 740| 890| 70295.0| 646.0| 781.0| 1167.0|
 2866| 85980| 1320| 1720| 2230| 147989.0| 774.0| 2556.0| 3534.0|
 5803| 243726| 690| 720| 860| 108454.0| 430.0| 915.0| 948.0|
 7340| 44040| 6090| 5790| 10050| 769004.0| 5008.0| 13286.0| 22632.0|
 1580| 9480| 4600| 3940| 4650| 516516.0| 4243.0| 10827.0| 22441.0|
 7754| 186096| 17830| 17690| 37960| 3371877.0| 19857.0| 99038.0| 102876.0|
 7833| 140994| 6540| 5430| 13390| 821176.0| 3925.0| 17757.0| 26519.0|
 7880| 47280| 3370| 2800| 7230| 364301.0| 2676.0| 11846.0| 18927.0|
 7982| 47892| 820| 740| 2050| 65573.0| 93.0| 171.0| 2689.0|
 7993| 287748| 46200| 43920| 107230| 4907166.0| 23015.0| 39827.0| 205526.0|
 833| 9996| 14000| 9490| 19940| 1497553.0| 7747.0| 12788.0| 32485.0|
+-------+------------+-------------------+------------------+-----------+------------------------+----------------------------+----------------------+-------------------+
only showing top 20 rows

Group the market data into buckets and count the number of farmer's markets in each bucket.

* Use `selectExpr` to transform the market data into labels that identify which zip group they belong to (we used `int(zip/10)` to group the tax data) call the new value `zipcode`.  `selectExpr` is short for "Select Expression" and can process similar operations to SQL statements.
* Group by the `zipcode` you just created, then `count` the groups.
* Use another `selectExpr` to transform the data, you only need to keep the `count` and the `zipcode as zip`.
* Store the results in a new dataset called `cleanedMarkets`.
* `show` `cleanedMarkets`

In [12]:
cleanedMarkets = markets\
  .selectExpr("*", "int(zip / 10) as zipcode")\
  .groupBy("zipcode")\
  .count()\
  .selectExpr("double(count) as count", "zipcode as zip")

cleanedMarkets.show()

+-----+----+
count| zip|
+-----+----+
 5.0|4900|
 2.0|7240|
 8.0|4818|
 1.0|9852|
 2.0|5300|
 5.0|2122|
 2.0|9900|
 1.0|8592|
 1.0|1580|
 1.0|3175|
 1.0|7754|
 2.0|6336|
 1.0|7833|
 1.0|7340|
 1.0|6466|
 4.0|6620|
 1.0|1342|
 2.0| 496|
 1.0|5156|
 1.0|4101|
+-----+----+
only showing top 20 rows

Join the two cleaned datasets into one dataset for analysis.

* Outer join `cleanedMarkets` to `summedTaxes` using `zip` and `zipcode` as the join variable.
* Name the resulting dataset `joined`.

In [14]:
joined = cleanedMarkets\
  .join(summedTaxes, cleanedMarkets["zip"] == summedTaxes["zipcode"], "outer")

* `display` the `joined` data - do you see the 'null' values?

In [16]:
display(joined)

count,zip,zipcode,sum(zipcode),sum(single_returns),sum(joint_returns),sum(numdep),sum(total_income_amount),sum(taxable_interest_amount),sum(net_capital_gains),sum(biz_net_income)
null,null,463,8334,930,840,980,82700.0,460.0,825.0,4409.0
2.0,496,496,17856,3290,3250,4150,357071.0,1540.0,5704.0,10969.0
null,null,833,9996,14000,9490,19940,1497553.0,7747.0,12788.0,32485.0
1.0,1342,1342,40260,4800,3680,5450,475283.0,2919.0,6476.0,13496.0
1.0,1580,1580,9480,4600,3940,4650,516516.0,4243.0,10827.0,22441.0
5.0,2122,2122,127320,94310,46450,120950,8381305.0,32598.0,71136.0,202711.0
null,null,2366,99372,31870,23150,43120,3395514.0,15435.0,40289.0,46188.0
null,null,2659,31908,370,450,570,45116.0,97.0,-20.0,596.0
null,null,2866,85980,1320,1720,2230,147989.0,774.0,2556.0,3534.0
1.0,3175,3175,57150,4250,4440,9490,495720.0,4784.0,7824.0,14127.0


MLLib doesn't allow null values.  These values came up as `null` in the join because there were no farmer's markets in that zip code "basket".  It makes sense to replace the `null` values with zeros.
* Use the `na` prefix to `fill` the empty cells with `0`.
* Name the resulting dataset `prepped` and `display` it.

In [18]:
prepped = joined.na.fill(0)
display(prepped)

count,zip,zipcode,sum(zipcode),sum(single_returns),sum(joint_returns),sum(numdep),sum(total_income_amount),sum(taxable_interest_amount),sum(net_capital_gains),sum(biz_net_income)
0.0,0,463,8334,930,840,980,82700.0,460.0,825.0,4409.0
2.0,496,496,17856,3290,3250,4150,357071.0,1540.0,5704.0,10969.0
0.0,0,833,9996,14000,9490,19940,1497553.0,7747.0,12788.0,32485.0
1.0,1342,1342,40260,4800,3680,5450,475283.0,2919.0,6476.0,13496.0
1.0,1580,1580,9480,4600,3940,4650,516516.0,4243.0,10827.0,22441.0
5.0,2122,2122,127320,94310,46450,120950,8381305.0,32598.0,71136.0,202711.0
0.0,0,2366,99372,31870,23150,43120,3395514.0,15435.0,40289.0,46188.0
0.0,0,2659,31908,370,450,570,45116.0,97.0,-20.0,596.0
0.0,0,2866,85980,1320,1720,2230,147989.0,774.0,2556.0,3534.0
1.0,3175,3175,57150,4250,4440,9490,495720.0,4784.0,7824.0,14127.0


### Part Two -Use MLLib with Spark
* Put all the features into a single vector.  
* Create an array to list the names of all the **non-feature** columns: `zip`, `zipcode`, `count`, call it `nonFeatureCols`.
* Create a list of names called `featureCols` which excludes the columns in `nonFeatureCols`.
* `print` the `featureCols`.

In [20]:
nonFeatureCols = {'zip', 'zipcode', 'count'}
featureCols = [column for column in prepped.columns if column not in nonFeatureCols]
print(featureCols)

['sum(zipcode)', 'sum(single_returns)', 'sum(joint_returns)', 'sum(numdep)', 'sum(total_income_amount)', 'sum(taxable_interest_amount)', 'sum(net_capital_gains)', 'sum(biz_net_income)']

* Use the `VectorAssembler` from `pyspark.ml.feature` to add a `features` vector to the `prepped` dataset.
* Call the new dataset `finalPrep`, then `display` only the `zipcode` and `features` from `finalPrep`.

In [22]:
from pyspark.ml.feature import VectorAssembler

assembler = VectorAssembler(
    inputCols=[column for column in featureCols],
    outputCol='features')

finalPrep = assembler.transform(prepped)
display(finalPrep.select('zipcode', 'features'))

zipcode,features
463,"List(1, 8, List(), List(8334.0, 930.0, 840.0, 980.0, 82700.0, 460.0, 825.0, 4409.0))"
496,"List(1, 8, List(), List(17856.0, 3290.0, 3250.0, 4150.0, 357071.0, 1540.0, 5704.0, 10969.0))"
833,"List(1, 8, List(), List(9996.0, 14000.0, 9490.0, 19940.0, 1497553.0, 7747.0, 12788.0, 32485.0))"
1342,"List(1, 8, List(), List(40260.0, 4800.0, 3680.0, 5450.0, 475283.0, 2919.0, 6476.0, 13496.0))"
1580,"List(1, 8, List(), List(9480.0, 4600.0, 3940.0, 4650.0, 516516.0, 4243.0, 10827.0, 22441.0))"
2122,"List(1, 8, List(), List(127320.0, 94310.0, 46450.0, 120950.0, 8381305.0, 32598.0, 71136.0, 202711.0))"
2366,"List(1, 8, List(), List(99372.0, 31870.0, 23150.0, 43120.0, 3395514.0, 15435.0, 40289.0, 46188.0))"
2659,"List(1, 8, List(), List(31908.0, 370.0, 450.0, 570.0, 45116.0, 97.0, -20.0, 596.0))"
2866,"List(1, 8, List(), List(85980.0, 1320.0, 1720.0, 2230.0, 147989.0, 774.0, 2556.0, 3534.0))"
3175,"List(1, 8, List(), List(57150.0, 4250.0, 4440.0, 9490.0, 495720.0, 4784.0, 7824.0, 14127.0))"


* Display the feature columns graphed out against each other as a scatter plot (hint: exclude `zip`, `zipcode` and `features` using `drop`)

In [24]:
display(finalPrep.drop("zip").drop("zipcode").drop("features"))

count,sum(zipcode),sum(single_returns),sum(joint_returns),sum(numdep),sum(total_income_amount),sum(taxable_interest_amount),sum(net_capital_gains),sum(biz_net_income)
0.0,8334,930,840,980,82700.0,460.0,825.0,4409.0
2.0,17856,3290,3250,4150,357071.0,1540.0,5704.0,10969.0
0.0,9996,14000,9490,19940,1497553.0,7747.0,12788.0,32485.0
1.0,40260,4800,3680,5450,475283.0,2919.0,6476.0,13496.0
1.0,9480,4600,3940,4650,516516.0,4243.0,10827.0,22441.0
5.0,127320,94310,46450,120950,8381305.0,32598.0,71136.0,202711.0
0.0,99372,31870,23150,43120,3395514.0,15435.0,40289.0,46188.0
0.0,31908,370,450,570,45116.0,97.0,-20.0,596.0
0.0,85980,1320,1720,2230,147989.0,774.0,2556.0,3534.0
1.0,57150,4250,4440,9490,495720.0,4784.0,7824.0,14127.0


* Split the `finalPrep` data set into training and testing subsets.  The sets should be randomly selected, 70 percent of the samples should go into the `training` set, and 30 percent should go into the `test` set.
* Cache `training` and `test`.
* Perform an action such as `count` to populate the cache.

In [26]:
(training, test) = finalPrep.randomSplit((0.7, 0.3))

training.cache()
test.cache()

print(training.count())
print(test.count())

4080
1722

Spark MLLib supports both `regressors` and `classifiers`, in this example you will use linear regression.  Once you create the `regressor` you will train it, and it will return a `Model`. The `Model` will be the object you use to make predictions.

* Create an instance of the `LinearRegression` algorithm called `lrModel`:
* Set the label column to "count"
* Set the features column to "features"
* Set the "ElasticNetParam" to 0.5 (this controlls the mix of l1 and l2 regularization--we'll just use an equal amount of each)
* Print the results of calling `explainParams` on `lrModel`.  This will show you all the possible parameters, and whether or not you have customized them.

In [28]:
from pyspark.ml.regression import LinearRegression

lrModel = LinearRegression()\
  .setLabelCol("count")\
  .setFeaturesCol("features")\
  .setElasticNetParam(0.5)

print("Printing out the model Parameters:")
print("-"*20)
print(lrModel.explainParams())
print("-"*20)

Printing out the model Parameters:
--------------------
aggregationDepth: suggested depth for treeAggregate (>= 2). (default: 2)
elasticNetParam: the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the penalty is an L2 penalty. For alpha = 1, it is an L1 penalty. (default: 0.0, current: 0.5)
epsilon: The shape parameter to control the amount of robustness. Must be > 1.0. Only valid when loss is huber (default: 1.35)
featuresCol: features column name. (default: features, current: features)
fitIntercept: whether to fit an intercept term. (default: True)
labelCol: label column name. (default: label, current: count)
loss: The loss function to be optimized. Supported options: squaredError, huber. (default: squaredError)
maxIter: max number of iterations (>= 0). (default: 100)
predictionCol: prediction column name. (default: prediction)
regParam: regularization parameter (>= 0). (default: 0.0)
solver: The solver algorithm for optimization. Supported options: auto, normal, l-bfgs. (default: auto)
standardization: whether to standardize the training features before fitting the model. (default: True)
tol: the convergence tolerance for iterative algorithms (>= 0). (default: 1e-06)
weightCol: weight column name. If this is not set or empty, we treat all instance weights as 1.0. (undefined)
--------------------

* Use the `fit` method on `lrModel` to provide the `training` dataset for fitting. 
* Store the results in `lrFitted`.

In [30]:
lrFitted = lrModel.fit(training)

* Make a prediction by using the `transform` method on `lrFitted`, passing it the `test` dataset. 
* Store the results in `holdout`.
* `transform` adds a new column called "prediction" to the data we passed into it.
* Display the `prediction` and `count` from `holdout`

In [32]:
holdout = lrFitted.transform(test)
display(holdout.select("prediction", "count"))

prediction,count
2.536069108577168,0.0
0.7805672359196321,0.0
0.6569546045481557,0.0
1.5003204793057077,1.0
1.5581101235202186,1.0
1.2972707203632114,1.0
1.0047101109598957,1.0
0.9296333303190338,1.0
1.5761137833736876,2.0
2.565416898087551,4.0


The `transform` method shows us how many farmer's markets the `lrFitted` method predicts there will be in each zip code based on the features we provided.  The raw predictions are not rounded at all.  

* Use a `selectExpr` to relabel `prediction` as `raw_prediction`.
* `round` the `prediction` and call it `prediction` inside the expression
* Select `count` for comparison purposes.
* Create a column called `equal` that will let us know if the model predicted correctly.

In [34]:
holdout = holdout.selectExpr(\
                             "prediction as raw_prediction", \
                             "double(round(prediction)) as prediction", \
                             "count", \
                             """CASE double(round(prediction)) = count 
                                WHEN true then 1
                                ELSE 0
                                END as equal""")
display(holdout)

raw_prediction,prediction,count,equal
2.536069108577168,3.0,0.0,0
0.7805672359196321,1.0,0.0,0
0.6569546045481557,1.0,0.0,0
1.5003204793057077,2.0,1.0,0
1.5581101235202186,2.0,1.0,0
1.2972707203632114,1.0,1.0,1
1.0047101109598957,1.0,1.0,1
0.9296333303190338,1.0,1.0,1
1.5761137833736876,2.0,2.0,1
2.565416898087551,3.0,4.0,0


* Use another `selectExpr` to `display` the proportion of predictions that were exactly correct.

In [36]:
display(holdout.selectExpr("sum(equal)/sum(1)"))

(sum(equal) / sum(1))
0.27526132404181186


* Use `RegressionMetrics` to get more insight into the model performance. NOTE: Regression metrics requires input formatted as tuples of `double`s where the first item is the `prediction` and the second item is the observation (in this case the observation is `count`).  Once you have `map`ped these values from `holdout` you can directly pass them to the `RegressionMetrics` constructor.

In [38]:
from pyspark.mllib.evaluation import RegressionMetrics

mapped = holdout.select("prediction", "count").rdd.map(lambda x: (float(x[0]), float(x[1])))
rm = RegressionMetrics(mapped)

print ("MSE: ", rm.meanSquaredError)
print ("MAE: ", rm.meanAbsoluteError)
print ("RMSE Squared: ", rm.rootMeanSquaredError)
print ("R Squared: ", rm.r2)
print ("Explained Variance: ", rm.explainedVariance)

MSE: 4431.511614401858
MAE: 2.648664343786295
RMSE Squared: 66.5695997764885
R Squared: -1608.4420644161876
Explained Variance: 4429.473818022153

Because these results still aren't very good, rather than training a single-model, let's train several using a pipeline.

* Use a `RandomForestRegressor` algorithm.  This algorithm has several `hyperparameters` that we can tune, rather than tune them individually, we will use a `ParamGridBuilder` to search the "hyperparameter space" for us.  This can take some time on small clusters, so be patient.

* Use the `Pipeline` to feed the algorithm into a `CrossValidator` to help prevent "overfitting".
* Use the `CrossValidator` uses a `RegressionEvaluator` to test the model results against a metric (default is RMSE).

* NOTE: In production, using AWS EC2 compute-optimized instance speed this up -- 3 min (c3.4xlarge) vs 10 min (r3.xlarge)

In [40]:
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import RegressionEvaluator

rfModel = RandomForestRegressor()\
  .setLabelCol("count")\
  .setFeaturesCol("features")
  
paramGrid = ParamGridBuilder()\
  .addGrid(rfModel.maxDepth, [5, 10])\
  .addGrid(rfModel.numTrees, [20, 60])\
  .build()

steps = [rfModel]

pipeline = Pipeline().setStages(steps)

cv = CrossValidator()\
  .setEstimator(pipeline)\
  .setEstimatorParamMaps(paramGrid)\
  .setEvaluator(RegressionEvaluator().setLabelCol("count"))

pipelineFitted = cv.fit(training)

* Access the best model on the `pipelineFitted` object by accessing the first stage of the `bestModel` attribute.

In [42]:
print("The Best Parameters:\n--------------------")
print(pipelineFitted.bestModel.stages[0])

The Best Parameters:
--------------------
RandomForestRegressionModel (uid=RandomForestRegressor_4ea2bf537e0bd8736de8) with 60 trees

* Use the `bestModel` to `transform` the `test` dataset.  
* Use a `selectExpr` to show the raw prediction, rounded prediction, count, and whether or not the prediction exactly matched (hint: this is the same `selectExpr` you used on the previous model results).
* Store the results in `holdout2`, then display.

In [44]:
holdout2 = pipelineFitted.bestModel\
  .transform(test)\
  .selectExpr("prediction as raw_prediction", \
    "double(round(prediction)) as prediction", \
    "count", \
    """CASE double(round(prediction)) = count 
  WHEN true then 1
  ELSE 0
END as equal""")
  
display(holdout2)

raw_prediction,prediction,count,equal
2.1208811165237704,2.0,0.0,0
1.034564160452733,1.0,0.0,0
0.3262088106675191,0.0,0.0,1
0.9946190918414828,1.0,1.0,1
1.1713894558245073,1.0,1.0,1
0.9660077203246353,1.0,1.0,1
0.23097336436447088,0.0,1.0,0
0.6233667508843822,1.0,1.0,1
2.2609459765207394,2.0,2.0,1
2.7339864234511397,3.0,4.0,0


* Show the `RegressionMetrics` for the new model results.

In [46]:
from pyspark.mllib.evaluation import RegressionMetrics

mapped2 = holdout2.select("prediction", "count").rdd.map(lambda x: (float(x[0]), float(x[1])))
rm2 = RegressionMetrics(mapped2)

print ("MSE: ", rm2.meanSquaredError)
print ("MAE: ", rm2.meanAbsoluteError)
print ("RMSE Squared: ", rm2.rootMeanSquaredError)
print ("R Squared: ", rm2.r2)
print ("Explained Variance: ", rm2.explainedVariance)

MSE: 5.454123112659698
MAE: 1.116144018583043
RMSE Squared: 2.3354064127384118
R Squared: -0.9808354182106516
Explained Variance: 4.398097787598085

* See if there an improvement in the "exactly right" proportion.

In [48]:
display(holdout2.selectExpr("sum(equal)/sum(1)"))

(sum(equal) / sum(1))
0.35772357723577236
